# Create AMITT website sql

Creates the sqlite file used as a datasource in the AMITT website tests. 

In [2]:
# Get AMITT variables

import pandas as pd
import sqlite3 as sql
import os
from sqlalchemy import create_engine
from generate_amitt_ttps import Amitt
pd.set_option('display.max_rows', 1000)


# Generate AMITT datasets
amitt = Amitt()

dbasetype = 'postgresql_local' #'sqlite' #'heroku'


# Create connection to database
if dbasetype == 'sqlite':
    # Generate minimal sqlite database from the Amitt variables
    conn = sql.connect('amittsite.sqlite')
elif dbasetype == 'postgresql_local':
    conn = create_engine('postgresql://sara:@localhost:5432/amittsite')

    
# Add table to database    
def add_table(dataframe, tablename, columns):    
    # Create sql-appropriate table from dataframe
    colnames = ', '.join(['{} TEXT NOT NULL'.format(col) for col in columns])
    newtable = dataframe[columns].copy().applymap(str)
    newtable['id'] = range(1,len(newtable)+1)
    
    # send to database
    if dbasetype == 'sqlite':
        conn.execute("DROP TABLE IF EXISTS {}".format(tablename))
        conn.execute('''CREATE TABLE {} (id INTEGER PRIMARY KEY AUTOINCREMENT, {});'''.format(tablename, colnames))
        newtable.to_sql(tablename, conn, index=False, if_exists='append')
        conn.commit()
    elif dbasetype == 'postgresql_local':
        newtable.to_sql(tablename, conn, index=False, if_exists='replace')
    return newtable


# Build a cross-table
def object_tactics_techniques(objectcol, objecttable, crosstable):
    # objects to techniques
    ctech = crosstable.copy()
    ctech = ctech[(ctech['technique_id'] != '') & (~ctech['technique_id'].str.startswith('TA'))]
    ctech.rename(columns={'amitt_id':objectcol}, inplace=True)
    ctech['summary'] = 'N/A'

    # objects to tactics
    ctact = crosstable[crosstable['technique_id'].str.startswith('TA')].copy()
    ctact.rename(columns={'amitt_id':objectcol, 'technique_id': 'tactic_id'}, inplace=True)
    ctact['main_tactic'] = 'N'
    ctactmain = objecttable[['amitt_id', 'tactic_id']].copy()
    ctactmain.rename(columns={'amitt_id':objectcol}, inplace=True)
    ctactmain['main_tactic'] = 'Y'
    ctact = pd.concat([ctact, ctactmain], ignore_index=True, sort=False)
    ctact['summary'] = 'N/A'
    return(ctech, ctact)

#Load all the tables - Heroku needs them in correct order... 

# -- frameworks --

#newtable = add_table(amitt.df_actortypes, 'actor_type', ['amitt_id', 'sector_id', 'framework_id', 'name', 'summary'])
newtable = add_table(amitt.df_counters, 'counter', ['amitt_id', 'tactic_id', 'metatechnique_id', 'name', 'summary'])
newtable = add_table(amitt.df_detections, 'detection', ['amitt_id', 'tactic_id', 'name', 'summary'])
newtable = add_table(amitt.df_frameworks, 'framework', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_metatechniques, 'metatechnique', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_phases, 'phase', ['amitt_id', 'name', 'rank', 'summary'])
newtable = add_table(amitt.df_playbooks, 'playbook', ['amitt_id', 'object_id', 'name', 'summary'])
newtable = add_table(amitt.df_resources, 'resource', ['amitt_id', 'name', 'summary', 'resource_type'])
newtable = add_table(amitt.df_responsetypes, 'responsetype', ['amitt_id', 'name', 'summary'])
#newtable = add_table(amitt.df_sector, 'sector', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_tactics, 'tactic', ['amitt_id', 'phase_id', 'name', 'rank', 'summary'])
newtable = add_table(amitt.df_tasks, 'task', ['amitt_id', 'tactic_id', 'framework_id', 'name', 'summary'])
newtable = add_table(amitt.df_techniques, 'technique', ['amitt_id', 'tactic_id', 'name', 'summary'])

(ctech, ctact) = object_tactics_techniques('counter_id', amitt.df_counters, amitt.cross_counterid_techniqueid)
newtable = add_table(ctech, 'counter_technique', ['counter_id', 'technique_id', 'summary'])
newtable = add_table(ctact, 'counter_tactic', ['counter_id', 'tactic_id', 'main_tactic', 'summary'])

(dtech, dtact) = object_tactics_techniques('detection_id', amitt.df_detections, amitt.cross_detectionid_techniqueid)
newtable = add_table(dtech, 'detection_technique', ['detection_id', 'technique_id', 'summary'])
newtable = add_table(dtact, 'detection_tactic', ['detection_id', 'tactic_id', 'main_tactic', 'summary'])

# -- datasets --

newtable = add_table(amitt.df_examples, 'example', ['amitt_id', 'object_id', 'name', 'summary'])

# dataset
# reference - create this from other tables. 
# Also need incidentcounter etc - create from data
newtable = add_table(amitt.df_groups, 'group', ['amitt_id', 'name', 'url', 'summary', 
                                                'sector', 'primary_role', 'secondary_role', 
                                                'primary_subject', 'secondary_subject', 
                                                'volunteers', 'region', 'country', 
                                                'twitter_handle'])
newtable = add_table(amitt.df_incidents, 'incident', ['amitt_id', 'name', 'summary', 
                                                      'year_started', 'attributions_seen', 
                                                      'found_in_country', 'objecttype'])
newtable = add_table(amitt.df_tools, 'tool', ['amitt_id', 'name', 'summary',
                                              'organization', 'url', 'category', 
                                              'disinformation_use', 'cogseccollab_use', 
                                              'function', 'code_url', 'artifacts', 
                                              'automation', 'platform', 'accessibility'])
# incidenttechnique crosstable
it = vars(amitt)['it'][['amitt_id', 'name', 'summary', 'amitt_id_incident', 'amitt_id_technique']].copy()
it.rename(columns={'amitt_id_incident':'incident_id', 'amitt_id_technique': 'technique_id'}, inplace=True)
newtable = add_table(it, 'incident_technique', ['amitt_id', 'name', 'summary', 
                                                'incident_id', 'technique_id'])

# Load in users table, and close connection
if dbasetype == 'sqlite':
    conn.execute("DROP TABLE IF EXISTS {}".format('users'))
    conn.execute('''CREATE TABLE users (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT NOT NULL UNIQUE, password TEXT NOT NULL);''')
    conn.close()
else:
    dfusers = pd.DataFrame([['test','testing']], columns=['username', 'password'])
    usertable = add_table(dfusers, 'users', ['username', 'password'])

newtable

,amitt_id,name,summary,incident_id,technique_id,id
0,IT00000001,buy FB targeted ads,,I00002,T0018,1
1,IT00000010,Targeted FB paid ads,,I00005,T0018,2
2,IT00000057,Targeted FB paid ads,,I00017,T0018,3
3,IT00000002,"Promote ""funding"" campaign",,I00002,T0017,4
4,IT00000003,create web-site - information pollution,,I00002,T0019,5
5,IT00000014,RT & Sputnik generate information pollution,,I00005,T0019,6
6,IT00000031,RT & Sputnik generate information pollution (r...,,I00007,T0019,7
7,IT00000037,RT & Sputnik generate information pollution,,I00009,T0019,8
8,IT00000061,RT & Sputnik generate information pollution,,I00017,T0019,9
9,IT00000082,RT & Sputnik generate information pollution (s...,,I00029,T0019,10


## test area

In [ ]:
# Check which amitt variables we can see from here
print('{}\n'.format(vars(amitt).keys()))
testdfname = 'it'
testdf = vars(amitt)[testdfname]
print('{} columns: {}'.format(testdfname, testdf.columns))
testdf

In [ ]:
it = vars(amitt)['it'][['amitt_id', 'name', 'summary', 'amitt_id_incident', 'amitt_id_technique']].copy()
it.rename(columns={'amitt_id_incident':'incident_id', 
                  'amitt_id_technique': 'technique_id'}, inplace=True)
it

In [ ]:
dii = vars(amitt)['df_incidents']
dii['objecttype'].value_counts()

In [ ]:
dii[dii['objecttype'] == 'apt']

In [ ]:
dii = vars(amitt)['df_techniques']
dii

In [ ]:
# Generate arrays used to create D3 grids
dflists = dii.groupby('tactic_id')['amitt_id'].apply(list).reset_index()
dfidgrid = pd.DataFrame(dflists['amitt_id'].to_list())
dfgrid = pd.concat([dflists[['tactic_id']], dfidgrid], axis=1).fillna('')
gridarray = [dfgrid[col].to_list() for col in dfgrid.columns]
gridarray

In [ ]:
it.index = it.amitt_id
it[['name']].transpose().to_dict('records')[0]